In [1]:
import cpa
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as ad

[rank: 0] Global seed set to 0
2024-09-25 09:09:18,429	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-09-25 09:09:18,655	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_1.h5ad"
adata_test_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_1.h5ad"
adata_ood_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_1.h5ad"

In [3]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [4]:
try:
    data_path = 'combo_sciplex_prep_hvg_filtered.h5ad'
    adata = sc.read(data_path)
except:
    import gdown
    gdown.download('https://drive.google.com/uc?export=download&id=1RRV0_qYKGTvD3oCklKfoZQFYqKJy4l6t')
    data_path = 'combo_sciplex_prep_hvg_filtered.h5ad'
    adata = sc.read(data_path)

adata

adata.obs["index"] = adata.obs_names.values
adata.obs["index_adapted"] = adata.obs.apply(lambda x: x["index"][:-2], axis=1)
adata.obs.set_index("index_adapted", inplace=True)

adata_ood.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_ood.obs["condition_ID"] = adata.obs["condition_ID"]
adata_ood.obs["log_dose"] = adata.obs["log_dose"]
adata_ood.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_ood.obs["cell_type"] = adata.obs["cell_type"]

adata_test.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_test.obs["condition_ID"] = adata.obs["condition_ID"]
adata_test.obs["log_dose"] = adata.obs["log_dose"]
adata_test.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_test.obs["cell_type"] = adata.obs["cell_type"]

adata_train.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_train.obs["condition_ID"] = adata.obs["condition_ID"]
adata_train.obs["log_dose"] = adata.obs["log_dose"]
adata_train.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_train.obs["cell_type"] = adata.obs["cell_type"]

In [104]:
control_cells_gex = adata_ood[adata_ood.obs["condition"]=="control"].X.A
obs_names = list(adata_ood[adata_ood.obs["condition"]=="control"].obs_names)

all_data = []
obs_df = pd.DataFrame(columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
indices = []

for condition in adata_train.obs["condition"].cat.categories:
    if condition=="control":
        continue
    all_data.append(control_cells_gex)
    filtered_row = pd.Series(adata_train[adata_train.obs["condition"] == condition].obs[["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"]].iloc[0].values)
    filtered_row_df = pd.DataFrame([filtered_row.values], columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
    duplicated_rows = pd.concat([filtered_row_df] * 500, ignore_index=True)
    obs_df = pd.concat([obs_df, duplicated_rows], ignore_index=True)

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)


# Create the Anndata object
adata_ood_new = ad.AnnData(X=all_data_array, obs=obs_df)
adata_ood_new.var = adata_ood.var


control_cells_gex = adata_test[adata_test.obs["condition"]=="control"].X.A
obs_names = list(adata_test[adata_test.obs["condition"]=="control"].obs_names)

all_data = []
obs_df = pd.DataFrame(columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
indices = []

for condition in adata_test.obs["condition"].cat.categories:
    if condition=="control":
        continue
    all_data.append(control_cells_gex)
    filtered_row = pd.Series(adata_test[adata_test.obs["condition"] == condition].obs[["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"]].iloc[0].values)
    filtered_row_df = pd.DataFrame([filtered_row.values], columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
    duplicated_rows = pd.concat([filtered_row_df] * 500, ignore_index=True)
    obs_df = pd.concat([obs_df, duplicated_rows], ignore_index=True)

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)


# Create the Anndata object
adata_test_new = ad.AnnData(X=all_data_array, obs=obs_df)
adata_test_new.var = adata_test.var

In [106]:
import anndata

adata_combined = anndata.concat((adata_train, adata_test_new, adata_ood_new), label="split", keys=["train", "test", "ood"], join="inner")

In [107]:
adata_combined

AnnData object with n_obs × n_vars = 68698 × 2000
    obs: 'condition', 'cell_type', 'condition_ID', 'log_dose', 'smiles_rdkit', 'split'

In [108]:
adata_test

AnnData object with n_obs × n_vars = 2900 × 2000
    obs: 'sample', 'Size_Factor', 'n.umi', 'RT_well', 'Drug1', 'Drug2', 'Well', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'condition', 'pathway1', 'pathway2', 'split', 'control', 'cell_type', 'cell_line', 'smiles_drug_1', 'smiles_drug_2', 'ood_1', 'ood_2', 'ood_3', 'ood_4', 'cov_drug_dose', 'condition_ID', 'log_dose', 'smiles_rdkit'
    var: 'id', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'Drug1_colors', 'Drug2_colors', 'Well_colors', 'condition_colors', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'ood_1_colors', 'ood_2_colors', 'ood_3_colors', 'ood_4_colors', 'pathway1_colors', 'pathway2_colors', 'pathway_colors', 'pca', 'rank_genes_groups', 'rank_genes_groups_cov_all', 'split_colors', 'umap'
    obsm: 'X_pca', 'X_umap', 'ecfp_drug_

In [109]:
adata_test_new

AnnData object with n_obs × n_vars = 12000 × 2000
    obs: 'condition_ID', 'log_dose', 'smiles_rdkit', 'cell_type', 'condition'
    var: 'id', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'

In [110]:
adata_ood_new

AnnData object with n_obs × n_vars = 12000 × 2000
    obs: 'condition_ID', 'log_dose', 'smiles_rdkit', 'cell_type', 'condition'

In [111]:
adata_ood

AnnData object with n_obs × n_vars = 16280 × 2000
    obs: 'sample', 'Size_Factor', 'n.umi', 'RT_well', 'Drug1', 'Drug2', 'Well', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'condition', 'pathway1', 'pathway2', 'split', 'control', 'cell_type', 'cell_line', 'smiles_drug_1', 'smiles_drug_2', 'ood_1', 'ood_2', 'ood_3', 'ood_4', 'cov_drug_dose', 'condition_ID', 'log_dose', 'smiles_rdkit'
    uns: 'rank_genes_groups_cov_all'
    obsm: 'X_pca', 'X_umap', 'ecfp_drug_1', 'ecfp_drug_2'
    layers: 'counts'

In [112]:
frac_valid = adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs / (
    adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs + adata[adata.obs["split_1ct_MEC"] == "train"].n_obs
)

In [113]:
import numpy as np


def create_split(x):
    if x["split"] != "train":
        return x["split"]
    is_train = np.random.choice(2, p=[frac_valid, 1 - frac_valid])
    if is_train:
        return "train_train"
    return "train_valid"


adata_combined.obs["split_1ct_MEC"] = adata_combined.obs.apply(create_split, axis=1)

In [114]:
cpa.CPA.setup_anndata(adata_combined,
                      perturbation_key='condition_ID',
                      dosage_key='log_dose',
                      control_group='CHEMBL504',
                      batch_key=None,
                      is_count_data=False,
                      categorical_covariate_keys=['cell_type'],
                      deg_uns_cat_key='rank_genes_groups_cov',
                      max_comb_len=2,
                     )

100%|██████████| 68698/68698 [00:00<00:00, 945698.99it/s]


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [115]:
ae_hparams = {
    "n_latent": 128,
    "recon_loss": "gauss",
    "doser_type": "logsigm",
    "n_hidden_encoder": 512,
    "n_layers_encoder": 3,
    "n_hidden_decoder": 512,
    "n_layers_decoder": 3,
    "use_batch_norm_encoder": True,
    "use_layer_norm_encoder": False,
    "use_batch_norm_decoder": True,
    "use_layer_norm_decoder": False,
    "dropout_rate_encoder": 0.1,
    "dropout_rate_decoder": 0.1,
    "variational": False,
    "seed": 434,
}

trainer_params = {
    "n_epochs_kl_warmup": None,
    "n_epochs_pretrain_ae": 30,
    "n_epochs_adv_warmup": 50,
    "n_epochs_mixup_warmup": 3,
    "mixup_alpha": 0.1,
    "adv_steps": 2,
    "n_hidden_adv": 64,
    "n_layers_adv": 2,
    "use_batch_norm_adv": True,
    "use_layer_norm_adv": False,
    "dropout_rate_adv": 0.3,
    "reg_adv": 20.0,
    "pen_adv": 20.0,
    "lr": 0.0003,
    "wd": 4e-07,
    "adv_lr": 0.0003,
    "adv_wd": 4e-07,
    "adv_loss": "cce",
    "doser_lr": 0.0003,
    "doser_wd": 4e-07,
    "do_clip_grad": False,
    "gradient_clip_value": 1.0,
    "step_size_lr": 45,
}

In [116]:
model = cpa.CPA(
    adata=adata_combined,
    split_key="split_1ct_MEC",
    train_split="train_train",
    valid_split="train_valid",
    **ae_hparams,
)

[rank: 0] Global seed set to 434


In [117]:
model.train(
    max_epochs=2000,
    use_gpu=True,
    batch_size=128,
    plan_kwargs=trainer_params,
    early_stopping_patience=10,
    check_val_every_n_epoch=5,
    save_path="/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/1",
)

100%|██████████| 25/25 [00:00<00:00, 187.78it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 5/2000:   0%|          | 4/2000 [01:18<10:13:18, 18.44s/it, v_num=1, recon=-2.52e+3, r2_mean=0.789, adv_loss=2.31, acc_pert=0.186]


Epoch 00004: cpa_metric reached. Module best state updated.


Epoch 10/2000:   0%|          | 9/2000 [02:49<9:53:42, 17.89s/it, v_num=1, recon=-3.03e+3, r2_mean=0.793, adv_loss=2.22, acc_pert=0.196, val_recon=-2.71e+3, disnt_basal=0.0764, disnt_after=0.106, val_r2_mean=0.789, val_KL=nan]


Epoch 00009: cpa_metric reached. Module best state updated.


...


disnt_basal = 0.06474958824723316
disnt_after = 0.08087105271348542
val_r2_mean = 0.8013363856567465
val_r2_var = 0.4700556216192699
Epoch 30/2000:   1%|▏         | 29/2000 [09:01<9:57:37, 18.19s/it, v_num=1, recon=-3.56e+3, r2_mean=0.804, adv_loss=2.12, acc_pert=0.231, val_recon=-3.32e+3, disnt_basal=0.0625, disnt_after=0.0759, val_r2_mean=0.802, val_KL=nan] 
disnt_basal = 0.05893953390797288
disnt_after = 0.07260594276990029
val_r2_mean = 0.804453662545331
val_r2_var = 0.46275064591066367
Epoch 40/2000:   2%|▏         | 39/2000 [11:55<9:21:55, 17.19s/it, v_num=1, recon=-3.65e+3, r2_mean=0.805, adv_loss=2.17, acc_pert=0.226, val_recon=-3.39e+3, disnt_basal=0.0595, disnt_after=0.0738, val_r2_mean=0.801, val_KL=nan] 
disnt_basal = 0.059677064087185726
disnt_after = 0.07238881559434354
val_r2_mean = 0.8067639842285426
val_r2_var = 0.46642863165983345



Epoch 00039: cpa_metric reached. Module best state updated.


Epoch 45/2000:   2%|▏         | 44/2000 [13:19<8:58:20, 16.51s/it, v_num=1, recon=-3.7e+3, r2_mean=0.805, adv_loss=2.22, acc_pert=0.213, val_recon=-3.36e+3, disnt_basal=0.0597, disnt_after=0.0724, val_r2_mean=0.807, val_KL=nan] 


Epoch 00044: cpa_metric reached. Module best state updated.


Epoch 50/2000:   2%|▏         | 49/2000 [14:45<9:09:51, 16.91s/it, v_num=1, recon=-3.73e+3, r2_mean=0.806, adv_loss=2.3, acc_pert=0.203, val_recon=-3.43e+3, disnt_basal=0.0617, disnt_after=0.0758, val_r2_mean=0.808, val_KL=nan] 
disnt_basal = 0.053076588104638875
disnt_after = 0.06718163478335769
val_r2_mean = 0.801917647669091
val_r2_var = 0.4687775820232528
Epoch 60/2000:   3%|▎         | 59/2000 [17:33<8:58:06, 16.63s/it, v_num=1, recon=-3.77e+3, r2_mean=0.805, adv_loss=2.44, acc_pert=0.18, val_recon=-3.49e+3, disnt_basal=0.0499, disnt_after=0.0627, val_r2_mean=0.8, val_KL=nan]   
disnt_basal = 0.05071536134162378
disnt_after = 0.06369496800991234
val_r2_mean = 0.8086378677214001
val_r2_var = 0.4733516411142794
Epoch 70/2000:   3%|▎         | 69/2000 [20:23<8:56:04, 16.66s/it, v_num=1, recon=-3.82e+3, r2_mean=0.807, adv_loss=2.71, acc_pert=0.148, val_recon=-3.52e+3, disnt_basal=0.0533, disnt_after=0.0651, val_r2_mean=0.81, val_KL=nan] 
disnt_basal = 0.04584508663687847
disnt_after =

In [118]:
model.predict(adata_combined)

100%|██████████| 2147/2147 [00:09<00:00, 235.08it/s]


In [119]:
adata_pred_ood = adata_combined[adata_combined.obs["split"]=="ood"]

In [123]:
from sklearn.metrics import r2_score

r_squared = {}
for cond in adata_train.obs["condition"].cat.categories:
    if cond == "control":
        continue
    true = adata_train[adata_train.obs["condition"] == cond].X.toarray()
    pred = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["CPA_pred"]
    r_squared[cond] = r2_score(true.mean(axis=0), pred.mean(axis=0))

In [124]:
r_squared

{'Alvespimycin+Pirarubicin': 0.8882224424819912,
 'Cediranib+PCI-34051': 0.9861838729780792,
 'Dacinostat+Danusertib': 0.7627357503658885,
 'Dacinostat+Dasatinib': 0.769219546455608,
 'Dacinostat+PCI-34051': 0.7291066467448709,
 'Givinostat+Carmofur': 0.9554822382215329,
 'Givinostat+Cediranib': 0.9604173486307267,
 'Givinostat+Crizotinib': 0.9566146961030413,
 'Givinostat+Curcumin': 0.9696024993138019,
 'Givinostat+SRT2104': 0.9677737371942441,
 'Givinostat+Sorafenib': 0.8691524414423516,
 'Givinostat+Tanespimycin': 0.9159718085072399,
 'Panobinostat+Alvespimycin': 0.40797271184027717,
 'Panobinostat+Crizotinib': 0.42119712030343537,
 'Panobinostat+Curcumin': 0.4978276164386215,
 'Panobinostat+SRT1720': 0.5827845797232246,
 'Panobinostat+SRT3025': 0.48049134737600085,
 'Panobinostat+Sorafenib': 0.4773906528616585,
 'SRT2104+Alvespimycin': 0.9165665787205544,
 'control+Alvespimycin': 0.8958570412410288,
 'control+Dacinostat': 0.8144171117998322,
 'control+Givinostat': 0.963404763565255

In [125]:
np.mean(list(r_squared.values()))

0.7792372924325481

In [20]:
#adata_combined.write(
#    "/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/adata_with_predictions_1.h5ad"
#)

In [81]:
adata_pred_ood = adata_combined[adata_combined.obs["split"]=="ood"]

In [87]:
adata_pred_ood.obs["condition"]#.value_counts()

0       <function get_new_cond at 0x7f891e41ae50>
1       <function get_new_cond at 0x7f891e41ae50>
2       <function get_new_cond at 0x7f891e41ae50>
3       <function get_new_cond at 0x7f891e41ae50>
4       <function get_new_cond at 0x7f891e41ae50>
                          ...                    
3495    <function get_new_cond at 0x7f891e41ae50>
3496    <function get_new_cond at 0x7f891e41ae50>
3497    <function get_new_cond at 0x7f891e41ae50>
3498    <function get_new_cond at 0x7f891e41ae50>
3499    <function get_new_cond at 0x7f891e41ae50>
Name: condition, Length: 3500, dtype: object

In [82]:
from sklearn.metrics import r2_score

r_squared = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    true = adata_train[adata_train.obs["condition"] == cond].X.toarray()
    pred = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["CPA_pred"]
    r_squared[cond] = r2_score(true.mean(axis=0), pred.mean(axis=0))

ValueError: Input contains NaN.

In [83]:
true

array([], shape=(0, 2000), dtype=float32)

In [28]:
np.mean(list(r_squared.values()))

0.7611490801416471

In [72]:
adata_pred_train = adata_combined[adata_combined.obs["split"]=="train"]

In [77]:
adata_ood.obs["condition"].value_counts()

condition
SRT3025+Cediranib         3016
Givinostat+Dasatinib      2421
control+Dasatinib         2343
Givinostat+SRT1720        2260
Panobinostat+SRT2104      1971
Panobinostat+Dasatinib    1955
Panobinostat+PCI-34051    1814
control                    500
Name: count, dtype: int64

In [78]:
d = {"SRT3025+Cediranib": 'Alvespimycin+Pirarubicin',
    "Givinostat+Dasatinib": 'Cediranib+PCI-34051',
    "control+Dasatinib": 'Dacinostat+Danusertib',
    "Givinostat+SRT1720": 'Givinostat+Cediranib',
    "Panobinostat+SRT2104": 'Givinostat+Crizotinib',}

In [89]:
def get_new_cond(x):
    if x["split"] == "ood":
        return d[x["condition"]]
    return x["condition"]
    
adata_combined.obs["condition"] = adata_combined.obs.apply(get_new_cond, axis=1)

KeyError: <function get_new_cond at 0x7f891e41ae50>

In [88]:
adata_combined.obs["condition"].value_counts()

condition
<function get_new_cond at 0x7f891e41ae50>    60198
Name: count, dtype: int64

In [75]:
from sklearn.metrics import r2_score

r_squared = {}
for cond in adata_train.obs["condition"].cat.categories:
    if cond == "control":
        continue
    true = adata_train[adata_train.obs["condition"] == cond].X.toarray()
    pred = adata_pred_train[adata_pred_train.obs["condition"] == cond].obsm["CPA_pred"]
    r_squared[cond] = r2_score(true.mean(axis=0), pred.mean(axis=0))

In [76]:
r_squared

{'Alvespimycin+Pirarubicin': 0.9694526778461512,
 'Cediranib+PCI-34051': 0.9830298396525267,
 'Dacinostat+Danusertib': 0.9903210866045268,
 'Dacinostat+Dasatinib': 0.9860534611641534,
 'Dacinostat+PCI-34051': 0.9933386653126748,
 'Givinostat+Carmofur': 0.9931940079028913,
 'Givinostat+Cediranib': 0.9914694289374953,
 'Givinostat+Crizotinib': 0.9941434229824847,
 'Givinostat+Curcumin': 0.9932098804340027,
 'Givinostat+SRT2104': 0.9920443313683595,
 'Givinostat+Sorafenib': 0.9844343704247122,
 'Givinostat+Tanespimycin': 0.9803631179783479,
 'Panobinostat+Alvespimycin': 0.9525659486259968,
 'Panobinostat+Crizotinib': 0.9868207245431262,
 'Panobinostat+Curcumin': 0.9930675281987964,
 'Panobinostat+SRT1720': 0.9942105373102229,
 'Panobinostat+SRT3025': 0.9927954532641882,
 'Panobinostat+Sorafenib': 0.9915281858609459,
 'SRT2104+Alvespimycin': 0.957734895215754,
 'control+Alvespimycin': 0.9620305669151762,
 'control+Dacinostat': 0.9901087711046277,
 'control+Givinostat': 0.9901133218192212,


In [62]:
params = list(ae_hparams.keys())
params.append("r_squared")

In [63]:
params = list(ae_hparams.keys())
params.append("r_squared")
df_all = pd.DataFrame(index=params)

In [65]:
df_all

""
n_latent
recon_loss
doser_type
n_hidden_encoder
n_layers_encoder
n_hidden_decoder
n_layers_decoder
use_batch_norm_encoder
use_layer_norm_encoder
use_batch_norm_decoder


In [41]:
ae_hparams.update(trainer_params)


In [53]:
df = pd.DataFrame(ae_hparams, index=["params"]).T
df.loc["r_squared"] = np.mean(list(r_squared.values()))

In [54]:
df

,params
n_latent,128
recon_loss,gauss
doser_type,logsigm
n_hidden_encoder,512
n_layers_encoder,3
n_hidden_decoder,512
n_layers_decoder,3
use_batch_norm_encoder,True
use_layer_norm_encoder,False
use_batch_norm_decoder,True


In [46]:
df_2 = pd.DataFrame(ae_hparams, index=["params"]).T

In [47]:
df_2

,params
n_latent,128
recon_loss,gauss
doser_type,logsigm
n_hidden_encoder,512
n_layers_encoder,3
n_hidden_decoder,512
n_layers_decoder,3
use_batch_norm_encoder,True
use_layer_norm_encoder,False
use_batch_norm_decoder,True


In [48]:
df_2.loc["step_size_lr"] = 2

In [49]:
df_2

,params
n_latent,128
recon_loss,gauss
doser_type,logsigm
n_hidden_encoder,512
n_layers_encoder,3
n_hidden_decoder,512
n_layers_decoder,3
use_batch_norm_encoder,True
use_layer_norm_encoder,False
use_batch_norm_decoder,True


In [51]:
df["params_2"] = df_2["params"]

In [52]:
df

,params,params_2
n_latent,128,128
recon_loss,gauss,gauss
doser_type,logsigm,logsigm
n_hidden_encoder,512,512
n_layers_encoder,3,3
n_hidden_decoder,512,512
n_layers_decoder,3,3
use_batch_norm_encoder,True,True
use_layer_norm_encoder,False,False
use_batch_norm_decoder,True,True


In [66]:
trainer_params = {
    "n_epochs_kl_warmup": None,
    "n_epochs_pretrain_ae": 30,
    "n_epochs_adv_warmup": 50,
    "n_epochs_mixup_warmup": 3,
    "mixup_alpha": 0.1,
    "adv_steps": 2,
    "n_hidden_adv": 64,
    "n_layers_adv": 2,
    "use_batch_norm_adv": True,
    "use_layer_norm_adv": False,
    "dropout_rate_adv": 0.3,
    "reg_adv": 20.0,
    "pen_adv": 20.0,
    "lr": 0.0003,
    "wd": 4e-07,
    "adv_lr": 0.0003,
    "adv_wd": 4e-07,
    "adv_loss": "cce",
    "doser_lr": 0.0003,
    "doser_wd": 4e-07,
    "do_clip_grad": False,
    "gradient_clip_value": 1.0,
    "step_size_lr": 45,
}

In [90]:
n_epochs_pretrain_ae = [30, 50]
n_epochs_adv_warmup = [50, 100]
n_epochs_mixup_warmup = [3, 10]
mixup_alpha = [0.1, 0.5]
adv_steps = [None, 2]
step_size_lr = [10, 45]

In [94]:
import itertools

combinations_dicts = [
    {
        'n_epochs_pretrain_ae': ae,
        'n_epochs_adv_warmup': adv_warmup,
        'n_epochs_mixup_warmup': mixup_warmup,
        'mixup_alpha': alpha,
        'adv_steps': adv,
        'step_size_lr': lr
    }
    for ae, adv_warmup, mixup_warmup, alpha, adv, lr in itertools.product(
        n_epochs_pretrain_ae,
        n_epochs_adv_warmup,
        n_epochs_mixup_warmup,
        mixup_alpha,
        adv_steps,
        step_size_lr
    )
]

In [97]:
combinations_dicts[1]

{'n_epochs_pretrain_ae': 30,
 'n_epochs_adv_warmup': 50,
 'n_epochs_mixup_warmup': 3,
 'mixup_alpha': 0.1,
 'adv_steps': None,
 'step_size_lr': 45}

In [98]:
combinations_dicts[0]

{'n_epochs_pretrain_ae': 30,
 'n_epochs_adv_warmup': 50,
 'n_epochs_mixup_warmup': 3,
 'mixup_alpha': 0.1,
 'adv_steps': None,
 'step_size_lr': 10}

In [101]:
trainer_params.update(combinations_dicts[1])

In [102]:
trainer_params

{'n_epochs_kl_warmup': None,
 'n_epochs_pretrain_ae': 30,
 'n_epochs_adv_warmup': 50,
 'n_epochs_mixup_warmup': 3,
 'mixup_alpha': 0.1,
 'adv_steps': None,
 'n_hidden_adv': 64,
 'n_layers_adv': 2,
 'use_batch_norm_adv': True,
 'use_layer_norm_adv': False,
 'dropout_rate_adv': 0.3,
 'reg_adv': 20.0,
 'pen_adv': 20.0,
 'lr': 0.0003,
 'wd': 4e-07,
 'adv_lr': 0.0003,
 'adv_wd': 4e-07,
 'adv_loss': 'cce',
 'doser_lr': 0.0003,
 'doser_wd': 4e-07,
 'do_clip_grad': False,
 'gradient_clip_value': 1.0,
 'step_size_lr': 45}

In [127]:
i=0
df = pd.DataFrame(trainer_params, index=[f"params_{i}"]).T
df.loc["r_squared"] = np.mean(list(r_squared.values()))
df_res[f"params_{i}"] = df[f"params_{i}"]


NameError: name 'df_res' is not defined

In [128]:
params = list(ae_hparams.keys())
params.append("r_squared")
df_res = pd.DataFrame(index=params)

In [129]:
df_res

""
n_latent
recon_loss
doser_type
n_hidden_encoder
n_layers_encoder
n_hidden_decoder
n_layers_decoder
use_batch_norm_encoder
use_layer_norm_encoder
use_batch_norm_decoder


In [130]:
df

,params_0
n_epochs_kl_warmup,None
n_epochs_pretrain_ae,30
n_epochs_adv_warmup,50
n_epochs_mixup_warmup,3
mixup_alpha,0.1
adv_steps,2
n_hidden_adv,64
n_layers_adv,2
use_batch_norm_adv,True
use_layer_norm_adv,False
